# Market Sentiment Analyzer

A simple LangChain pipeline to analyze market sentiment for a company using real-time news. It:
- Accepts a company name (e.g., 'Microsoft' or 'Apple Inc' or 'Tredence').
- Fetches the stock ticker via web search (DuckDuckGo API).
- Retrieves news using Yahoo Finance (for public companies) or web search (for private companies).
- Analyzes sentiment with Azure OpenAI GPT-4o.
- Logs steps with MLflow and outputs structured JSON.

## Setup Instructions

1. **Install Dependencies**:
   ```bash
   pip install mlflow langchain langchain-core langchain-community langchain-openai yfinance requests
   ```

## Azure and MLflow API Configuration Steps
1. **Azure OpenAI Configuration**:
   - Create an Azure OpenAI resource in the Azure portal.
   - Deploy the `gpt-4o` model (deployment name: `gpt-4o`).
   - Set environment variables:
     ```bash
     export AZURE_OPENAI_API_KEY="your-api-key"
     export AZURE_OPENAI_ENDPOINT="https://your-resource.openai.azure.com/"
     export AZURE_OPENAI_API_VERSION="2024-02-15-preview"
     ```
2. **MLflow Configuration**:
   - Ensure MLflow server is running at `http://20.75.92.162:5000`.
   - Start locally if needed:
     ```bash
     mlflow ui --host 0.0.0.0 --port 5000
     ```
   - Verify access to the MLflow UI to view logs (experiment: `raj_market_sentiment_analyzer`).

## Sample Command
- For a Python script, run:
  ```bash
  python sentiment_analyzer.py --company "Microsoft"
  ```
- In this notebook, execute the cells below sequentially, setting `company = "Microsoft"` in the last cell.

## Notes
- Handles public (e.g., Microsoft) and private (e.g., Tredence) companies.
- Uses `YahooFinanceNewsTool` for public companies and DuckDuckGo API for private ones.
- No static lookup or mock news.
- Logs to MLflow at `http://20.75.92.162:5000`.
- Output is JSON with fields: `company_name`, `stock_code`, `newsdesc`, `sentiment`, `people_names`, `places_names`, `other_companies_referred`, `related_industries`, `market_implications`, `confidence_score`.

In [94]:
!python -m pip install mlflow --quiet --upgrade
!python -m pip install langchain langchain-core langchain-community langchain-experimental yfinance requests --quiet
!python -m pip install langchain-openai --quiet

In [95]:
model_name='gpt4o'

from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(model=model_name)

In [96]:
import mlflow
mlflow.set_tracking_uri("http://20.75.92.162:5000")

In [97]:
import json
import mlflow
import requests
import yfinance as yf
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool

In [98]:
# Initialize tools
news_tool = YahooFinanceNewsTool()

#Helper Functions

In [99]:
def web_search(query, num_results=5):
    """Simple web search using a free API (e.g., SerpAPI alternative or direct)."""
    # Using a simple DuckDuckGo API wrapper without extra deps
    try:
        url = f"https://api.duckduckgo.com/?q={query}&format=json&no_html=1&skip_disambig=1"
        response = requests.get(url)
        data = response.json()
        if 'AbstractText' in data and data['AbstractText']:
            return data['AbstractText']
        elif 'RelatedTopics' in data:
            return ' '.join([t.get('Text', '') for t in data['RelatedTopics'][:3]])
        else:
            return "No results found."
    except Exception as e:
        return f"Search error: {str(e)}"

def get_stock_code(company_name):
    """Fetch stock ticker using web search and LLM."""
    with mlflow.start_run(nested=True, run_name="ticker_extraction"):
        search_result = web_search(f"{company_name} stock ticker symbol")
        prompt = ChatPromptTemplate.from_template(
            "Extract the stock ticker for {company_name} from: {search_result}. "
            "Return JSON with key 'ticker' (e.g., {{'ticker': 'MSFT'}}) or {{'ticker': 'N/A'}} if not found."
        )
        chain = prompt | model | JsonOutputParser()
        result = chain.invoke({"company_name": company_name, "search_result": search_result})
        ticker = result.get("ticker", "N/A")
        mlflow.log_param("company_name", company_name)
        mlflow.log_param("ticker", ticker)
        return ticker

In [100]:
def fetch_news(company_name, ticker):
    """Fetch news using Yahoo Finance or yf."""
    with mlflow.start_run(nested=True, run_name="news_fetching"):
        try:
            if ticker == "N/A":
                news = web_search(f"{company_name} recent news 2025", num_results=5)
            else:
                news = news_tool.run(ticker)
                # Fallback to yf if Yahoo Finance returns no news
                if not news or "No news found" in news:
                    ticker_obj = yf.Ticker(ticker)
                    news_items = ticker_obj.news[:3]  # Limit to 3
                    news_list = [
                        f"{item['content'].get('title', '')}: {item['content'].get('provider', {}).get('displayName', '')}"
                        for item in news_items if item.get('content', {}).get('title')
                    ]
                    news = "\n".join(news_list) if news_list else "No news found."
            news_summary = news[:500] + "..." if len(news) > 500 else news
            mlflow.log_param("news_summary", news_summary)
            return news_summary
        except Exception as e:
            mlflow.log_param("news_error", str(e))
            return f"No news found for {company_name}."

In [101]:
def analyze_sentiment(company_name, ticker, news):
    """Analyze news sentiment using LLM."""
    with mlflow.start_run(nested=True, run_name="raj_sentiment_analysis"):
        prompt = ChatPromptTemplate.from_template(
            """Analyze news for {company_name} ({ticker}) and return JSON with:
            - company_name: string
            - stock_code: string
            - newsdesc: string (news summary)
            - sentiment: string (Positive/Negative/Neutral)
            - people_names: list of strings
            - places_names: list of strings
            - other_companies_referred: list of strings
            - related_industries: list of strings
            - market_implications: string
            - confidence_score: float (0.0-1.0)
            
            News: {news}"""
        )
        chain = prompt | model | JsonOutputParser()
        result = chain.invoke({"company_name": company_name, "ticker": ticker, "news": news})
        mlflow.log_dict(result, "sentiment.json")
        return result

In [102]:
def sentiment_pipeline(company_name):
    """Main pipeline for sentiment analysis."""
    mlflow.set_experiment("raj_market_sentiment_analyzer")
    with mlflow.start_run(run_name=f"Sentiment_{company_name.replace(' ', '_')}"):
        ticker = get_stock_code(company_name)
        news = fetch_news(company_name, ticker)
        result = analyze_sentiment(company_name, ticker, news)
        return result

In [104]:
# Run the pipeline
company = "Microsoft"
result = sentiment_pipeline(company)
print(json.dumps(result, indent=2))

 View run ticker_extraction at: http://20.75.92.162:5000/#/experiments/279042531225761139/runs/0f2142bf059d4e43a17314c248439103
離 View experiment at: http://20.75.92.162:5000/#/experiments/279042531225761139
 View run news_fetching at: http://20.75.92.162:5000/#/experiments/279042531225761139/runs/b4ff5526f30141b487fb84ce88efc095
離 View experiment at: http://20.75.92.162:5000/#/experiments/279042531225761139
 View run raj_sentiment_analysis at: http://20.75.92.162:5000/#/experiments/279042531225761139/runs/c5c81f17b03c47e98522ce6e748fe7f2
離 View experiment at: http://20.75.92.162:5000/#/experiments/279042531225761139
 View run Sentiment_Microsoft at: http://20.75.92.162:5000/#/experiments/279042531225761139/runs/2da670861eb24e56b94088149fc1a69c
離 View experiment at: http://20.75.92.162:5000/#/experiments/279042531225761139
{
  "company_name": "Microsoft",
  "stock_code": "MSFT",
  "newsdesc": "The news highlights a comprehensive analysis of market trends involving Microsoft amidst 